In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits
import nltk; nltk.download('stopwords')
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import pandas as pd
hotel_review = pd.read_csv('Hotel_Reviews.csv')
hotel_review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397.0,1403.0,Only the park outside of the hotel was beauti...,11.0,7.0,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0.0,1403.0,No real complaints the hotel was great great ...,105.0,7.0,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42.0,1403.0,Location was good and staff were ok It is cut...,21.0,9.0,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210.0,1403.0,Great location in nice surroundings the bar a...,26.0,1.0,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140.0,1403.0,Amazing location and building Romantic setting,8.0,3.0,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
# 나라 중복값 확인
w_count  = {}
for lst in hotel_review['Reviewer_Nationality']:
    try: w_count[lst] += 1
    except: w_count[lst]=1
print(w_count)

# 유럽  = United Kingdom, Belgum, Norway, France, Italy, Netherlands, Germany, Denmark, Portugal, Sweden, Romaina, Greece, Bulgaria, Russia
# 아시아 = India, Malaysia, China, Turkey, Australia, New Zealand, Thailand, Singapore, Japan, South Korea, Taiwan, Hong Kong
# 중동, 아프리카 = Israel, United Arab Emirates, South Africa, Saudi Arabia, Bahrain, Kuwait
# 아메리카 = United Stated of America, Canada, Argentina, Brazil, Mexico

{' Russia ': 28, ' Ireland ': 282, ' Australia ': 351, ' United Kingdom ': 5159, ' New Zealand ': 43, ' Poland ': 29, ' Belgium ': 64, ' Norway ': 53, ' France ': 94, ' Italy ': 68, ' Canada ': 142, ' Netherlands ': 100, ' Germany ': 83, ' United States of America ': 547, ' Hungary ': 18, ' Israel ': 71, ' Switzerland ': 112, ' India ': 71, ' United Arab Emirates ': 137, ' Malaysia ': 40, ' Denmark ': 23, ' South Africa ': 57, ' Rwanda ': 1, ' Finland ': 25, ' Portugal ': 16, ' Luxembourg ': 19, ' Spain ': 57, ' Malta ': 19, ' China ': 49, ' Turkey ': 45, ' Ukraine ': 3, ' Gibraltar ': 1, ' Argentina ': 5, ' Abkhazia Georgia ': 4, ' Isle of Man ': 12, ' Sweden ': 52, ' Croatia ': 11, ' Saudi Arabia ': 128, ' Lithuania ': 4, ' Slovenia ': 2, ' Panama ': 2, ' Austria ': 25, ' Brazil ': 23, ' Taiwan ': 11, ' Iceland ': 13, ' Iran ': 5, ' Liechtenstein ': 1, ' Chile ': 3, ' Hong Kong ': 57, ' United States Minor Outlying Islands ': 3, ' Morocco ': 3, ' Oman ': 16, ' Lebanon ': 16, ' Qatar 

In [4]:
# 대륙별 행 추출
hotel_review_eu = hotel_review[['Reviewer_Nationality',"Negative_Review","Positive_Review"]]
Ru = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Russia "]
Un = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " United Kingdom "]
Be = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Belgum "]
No = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Norway "]
Fr = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " France "]
It = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Italy "]
Ne = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Netherlands "]
De = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Denmark "]
Po = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Portugal "]
Sw = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Sweden "]
RO = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Romaina "]
Gr = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Greece "]
Bu = hotel_review_eu[hotel_review_eu['Reviewer_Nationality']  == " Bulgaira "]
EU = pd.concat([Ru,Un,Be,No,Fr,It,Ne,De,Po,Sw,RO,Gr,Bu])
EU

,Reviewer_Nationality,Negative_Review,Positive_Review
0,Russia,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...
37,Russia,The glass wall separating the bathroom and th...,Very nice hotel located in a park Ca 30 minut...
191,Russia,Arena hotel is located in a historical buildi...,Location is good it takes 20 mins by feet and...
213,Russia,The room with stairs was quite uncomfortable ...,Very nice property stylish hotel friendly sta...
255,Russia,No Negative,Standard room is amazing location is adorable...
520,Russia,Housekeeping staff were rude shower had water...,Great food very nice bar staff
725,Russia,Got settled on the UNDERground floor without ...,No Positive
907,Russia,No Negative,Great location
1239,Russia,There is a definite problem with the smoking ...,Everything was great the service the rooms lo...
1471,Russia,Far from the tube,No Positive


In [0]:
Negative = EU.iloc[:,1]
Positive = EU.iloc[:,2]

In [6]:
Negative = [re.sub("\'", "", str(sent)) for sent in Negative]
Negative = [re.sub("No Negative'", "", str(sent)) for sent in Negative]
pprint(Negative[:1])

[' I am so angry that i made this post available via all possible sites i use '
 'when planing my trips so no one will make the mistake of booking this place '
 'I made my booking via booking com We stayed for 6 nights in this hotel from '
 '11 to 17 July Upon arrival we were placed in a small room on the 2nd floor '
 'of the hotel It turned out that this was not the room we booked I had '
 'specially reserved the 2 level duplex room so that we would have a big '
 'windows and high ceilings The room itself was ok if you don t mind the '
 'broken window that can not be closed hello rain and a mini fridge that '
 'contained some sort of a bio weapon at least i guessed so by the smell of it '
 'I intimately asked to change the room and after explaining 2 times that i '
 'booked a duplex btw it costs the same as a simple double but got way more '
 'volume due to the high ceiling was offered a room but only the next day SO i '
 'had to check out the next day before 11 o clock in order to ge

In [7]:
Positive = [re.sub("\'", "", str(sent1)) for sent1 in Positive]
Positive = [re.sub("No Positive'", "", str(sent1)) for sent1 in Positive]
pprint(Positive[:1])

[' Only the park outside of the hotel was beautiful ']


In [0]:
import gensim 
from gensim.models import Word2Vec

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words1 = list(sent_to_words(Negative))
data_words2 = list(sent_to_words(Positive))

print(data_words1[:1])
print(data_words2[:1])

[['am', 'so', 'angry', 'that', 'made', 'this', 'post', 'available', 'via', 'all', 'possible', 'sites', 'use', 'when', 'planing', 'my', 'trips', 'so', 'no', 'one', 'will', 'make', 'the', 'mistake', 'of', 'booking', 'this', 'place', 'made', 'my', 'booking', 'via', 'booking', 'com', 'we', 'stayed', 'for', 'nights', 'in', 'this', 'hotel', 'from', 'to', 'july', 'upon', 'arrival', 'we', 'were', 'placed', 'in', 'small', 'room', 'on', 'the', 'nd', 'floor', 'of', 'the', 'hotel', 'it', 'turned', 'out', 'that', 'this', 'was', 'not', 'the', 'room', 'we', 'booked', 'had', 'specially', 'reserved', 'the', 'level', 'duplex', 'room', 'so', 'that', 'we', 'would', 'have', 'big', 'windows', 'and', 'high', 'ceilings', 'the', 'room', 'itself', 'was', 'ok', 'if', 'you', 'don', 'mind', 'the', 'broken', 'window', 'that', 'can', 'not', 'be', 'closed', 'hello', 'rain', 'and', 'mini', 'fridge', 'that', 'contained', 'some', 'sort', 'of', 'bio', 'weapon', 'at', 'least', 'guessed', 'so', 'by', 'the', 'smell', 'of', 

In [10]:
%%time
model1 = Word2Vec(data_words1)

CPU times: user 967 ms, sys: 10.2 ms, total: 977 ms
Wall time: 579 ms


In [11]:
%%time
model2 = Word2Vec(data_words2)

CPU times: user 759 ms, sys: 6.85 ms, total: 766 ms
Wall time: 459 ms


In [36]:
# 토픽 간에 얼마나 연관성이 있는지
model1.wv.similarity('service', "room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.99197346

In [34]:
model1.wv.similarity('location', "room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.99249715

In [35]:
model1.wv.similarity('location', "travel")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9980682

In [37]:
# 부정적 토픽 연관단어 도출
model1.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('drinks', 0.9997082948684692),
 ('two', 0.9996693134307861),
 ('busy', 0.9996404051780701),
 ('terrible', 0.99962317943573),
 ('second', 0.9996143579483032),
 ('just', 0.9996017217636108),
 ('especially', 0.9995898008346558),
 ('restaurant', 0.9995889663696289),
 ('given', 0.9995888471603394),
 ('quality', 0.999587893486023)]

In [38]:
model1.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bathroom', 0.9991134405136108),
 ('bed', 0.999083399772644),
 ('and', 0.9984620809555054),
 ('quite', 0.998098611831665),
 ('hot', 0.9980074167251587),
 ('uncomfortable', 0.9977221488952637),
 ('bit', 0.9977128505706787),
 ('noisy', 0.9975695610046387),
 ('too', 0.9975110292434692),
 ('tiny', 0.9974052309989929)]

In [39]:
model1.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('quality', 0.9998161792755127),
 ('looking', 0.9997661113739014),
 ('lovely', 0.9997644424438477),
 ('busy', 0.9997528195381165),
 ('just', 0.9997406005859375),
 ('over', 0.9997373223304749),
 ('disappointing', 0.9997346997261047),
 ('around', 0.9997276663780212),
 ('standard', 0.9997270703315735),
 ('felt', 0.9996987581253052)]

In [40]:
model1.wv.most_similar("travel")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('your', 0.9990754723548889),
 ('wouldn', 0.9990749955177307),
 ('upgrade', 0.9990630149841309),
 ('who', 0.9990549087524414),
 ('manager', 0.9990535974502563),
 ('say', 0.9990510940551758),
 ('few', 0.9990449547767639),
 ('use', 0.9990352392196655),
 ('changed', 0.9990115761756897),
 ('while', 0.9990048408508301)]

In [41]:
# 긍정적 리뷰 토픽 연관단어 도출
model2.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('was', 0.9978694915771484),
 ('bathroom', 0.9972381591796875),
 ('facilities', 0.9955177307128906),
 ('breakfast', 0.9954632520675659),
 ('size', 0.9948987364768982),
 ('modern', 0.9948853254318237),
 ('beds', 0.9938538074493408),
 ('food', 0.9937745332717896),
 ('shower', 0.9936048984527588),
 ('quiet', 0.9930133819580078)]

In [45]:
model2.wv.most_similar("booking")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('their', 0.9997375011444092),
 ('her', 0.9996627569198608),
 ('work', 0.9996582865715027),
 ('without', 0.999646782875061),
 ('don', 0.999626636505127),
 ('what', 0.9996001124382019),
 ('better', 0.9995971918106079),
 ('thing', 0.999586284160614),
 ('ve', 0.9995829463005066),
 ('think', 0.9995749592781067)]

In [46]:
model2.wv.most_similar("travel")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('tourist', 0.999602735042572),
 ('through', 0.9995079040527344),
 ('theatre', 0.9994638562202454),
 ('by', 0.9994374513626099),
 ('see', 0.99942547082901),
 ('find', 0.9994229674339294),
 ('than', 0.9993686676025391),
 ('easily', 0.9993287324905396),
 ('st', 0.9993185997009277),
 ('show', 0.9992899894714355)]

In [47]:
model2.wv.most_similar("meal")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('he', 0.9993299841880798),
 ('both', 0.9992352724075317),
 ('look', 0.9992222785949707),
 ('expensive', 0.9992116689682007),
 ('bit', 0.9992042183876038),
 ('having', 0.9991991519927979),
 ('perfectly', 0.9991854429244995),
 ('young', 0.9991767406463623),
 ('deal', 0.9991666078567505),
 ('each', 0.9991433620452881)]